<a href="https://colab.research.google.com/github/sharma-kshitij-ks/Empowering-Investors-Hackathon/blob/main/EmpoweredinvestorsFinalCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize
import nltk
import openai  # Import the OpenAI library

nltk.download('punkt')
nltk.download("vader_lexicon")

# Initialize the Sentiment Intensity Analyzer
sia = SentimentIntensityAnalyzer()

# Set your OpenAI API key
openai.api_key = "Your_key"  # Replace with your actual API key

# Define the URL of the Yahoo Finance news section
url = "https://finance.yahoo.com/news"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all the news article elements on the page
article_elements = soup.find_all("li", class_="js-stream-content")

# Loop through the article elements and extract relevant information
for article_element in article_elements:
    article_title_element = article_element.find("h3")
    article_link_element = article_element.find("a", class_="js-content-viewer")

    if article_title_element and article_link_element:
        article_title = article_title_element.text.strip()
        article_link = article_link_element.get("href")

        # Visit the article URL and scrape the content
        article_url = f"https://finance.yahoo.com{article_link}"
        article_response = requests.get(article_url)
        article_soup = BeautifulSoup(article_response.content, "html.parser")

        article_content_element = article_soup.find("div", class_="caas-body")
        if article_content_element:
            article_text = article_content_element.get_text()
            sentences = sent_tokenize(article_text)

            # Initialize sentiment scores for ensemble
            total_sentiment_score = 0
            total_sentences = len(sentences)

            for sentence in sentences:
                # Perform sentiment analysis using Sentiment Intensity Analyzer
                sentiment_scores = sia.polarity_scores(sentence)
                sentiment_score = sentiment_scores["compound"]

                # Add sentiment score to ensemble total
                total_sentiment_score += sentiment_score

            # Calculate average sentiment score for ensemble
            average_sentiment_score = total_sentiment_score / total_sentences

            # Classify sentiment label based on ensemble average score
            if average_sentiment_score > 0.2:
                sentiment_label = "Genuine"
            elif average_sentiment_score < -0.2:
                sentiment_label = "Fake"
            else:
                sentiment_label = "Neutral"

            # Generate explanation using GPT
            prompt = f"Summarize why the news article titled '{article_title}' is {sentiment_label}."
            explanation = openai.Completion.create(
                engine="text-davinci-003",
                prompt=prompt,
                max_tokens=100
            )
            explanation_text = explanation.choices[0].text.strip()

            print("Title:", article_title)
            print("URL:", article_url)
            print("Sentiment Label:", sentiment_label)
            print("Average Sentiment Score:", average_sentiment_score)
            print("Explanation:", explanation_text)
            print("\n")